In [1]:
from citipy import citipy
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime
from config import weather_api_key

### Weather Data.

In [2]:
# Random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = list(zip(lats, lngs))

In [3]:
# Finding cities near the coordinates
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
# Check results
len(cities)

734

In [4]:
# API Url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

### API call

In [ ]:
# List of city data
city_data = []

# Print to logger
print("   Beginning Data Retrieval   ")
print("------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]       
            
            # Append the City information into city_data list
        city_data.append({"City": city.title(), 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city

    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete 
print("-----------------------------")
print("   Data Retrieval Complete   ")
print("-----------------------------")

   Beginning Data Retrieval   
------------------------------
Processing Record 1 of Set 1 | saskylakh
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | laguna
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | bredasdorp
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | nizhniy tsasuchey
Processing Record 8 of Set 1 | tevaitoa
Processing Record 9 of Set 1 | souillac
Processing Record 10 of Set 1 | saint anthony
Processing Record 11 of Set 1 | kununurra
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | airai
Processing Record 14 of Set 1 | coracora
Processing Record 15 of Set 1 | mar del plata
Processing Record 16 of Set 1 | butaritari
Processing Record 17 of Set 1 | vaini
Processing Record 18 of Set 1 | kolpny
Processing Record 19 of Set 1 | hobart
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 | kandrian
Processing Record 22 of Set 1 | makubetsu
Processing Record 23 of Set 1 | kutum

In [ ]:
# Converting the array of dictionaries to dataframe
city_data_df = pd.DataFrame(city_data)

In [12]:
# Check & clean up columns
print(city_data_df.columns)

new_column_order = ["City", "Country", "Lat",
                    "Lng", "Max Temp", "Humidity", 
                    "Cloudiness", "Wind Speed", 
                    "Current Description"] 

city_data_df = city_data_df[new_column_order]

print(city_data_df.dtypes)

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object


In [13]:
# Create csv
output_data_file = "Resources/city_database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")